In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import time
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

%matplotlib inline
#%matplotlib notebook

def cleanCapture(capture):
    captureRaw=capture
    capture=[]
    for each in captureRaw:
        if not np.isnan(each):
            capture.append(each)
    return capture
    
def bitVisDataSubADC(capture, NUM_CB=1, subADC=8, inv = True):
    res=[]
    resDiff=[]
    resBitSum=[]
    resBitSumDiff=[]
    capture=np.array(capture,dtype=int)
    if NUM_CB==None:
        SAMPLES=len(capture)
    else:
        SAMPLES=NUM_CB*256*64
    capture=capture[:SAMPLES]
    for captureSubADC in [capture[i::subADC] for i in range(subADC)]:
        countBits=[0]*8
        for item in [np.binary_repr(i, width=8) for i in captureSubADC]:
            if inv:
                countBits=[countBits[bit] + int(item[bit]) for bit in range(8)]
            else:
                countBits=[countBits[bit] + int(item[7-bit]) for bit in range(8)]
                
        res.append(countBits)
    for each in res:
        resBitSum.append([sum(each[:i+1]) for i in range(8)])
    for each in res:
        resDiff.append([item - int(SAMPLES/2/subADC) for item in each])
    for each in resBitSum:
        resBitSumDiff.append([each[i] - int(SAMPLES/2/subADC)*(i+1) for i in range(8)])
    return [res,resBitSum,resDiff,resBitSumDiff]

def plotPlainTD(fig=None, ax=None, capture=None, fname=None, SAMPLES=1024):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)
    avgCap = np.mean(capture)
    minCap = int(np.min(capture ))
    maxCap = int(np.max(capture ))
    SAMPLES=SAMPLES
    ax.set_ylim(-128,128)
    for k in range(-128,128,8):
        ax.plot([0,SAMPLES],[k,k], 'k-', alpha=.1, linewidth=1)
    ln=[]
    ln,    = ax.plot(capture[:SAMPLES],'b.',markersize=5, alpha=.8)
    lnAvg, = ax.plot([0,SAMPLES],[avgCap,avgCap], 'r--', alpha=.5, linewidth=1)
    lnMin, = ax.plot([0,SAMPLES],[minCap,minCap], 'y--', alpha=.8, linewidth=1)
    lnMax, = ax.plot([0,SAMPLES],[maxCap,maxCap], 'y--', alpha=.8, linewidth=1)
    lnZero, = ax.plot([0,SAMPLES],[0,0], 'g--', alpha=.8, linewidth=1)
    textstr = 'avg:{0:6.2f} min:{1:d} max:{2:d}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.65, textstr, transform=ax.transAxes, fontsize='large', verticalalignment='top', bbox=props)
    ax.set_title(fname, fontsize='large')
    ax.yaxis.set_major_locator(plt.MultipleLocator(8))
    ax.xaxis.set_major_locator(plt.MultipleLocator(64))
    if ax == None:
        return fig
    else:
        return ax

def plotTD(fig=None, ax=None, capture=None, fname=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)
    avgCap = np.mean(capture)
    minCap = np.min(capture )
    maxCap = np.max(capture )
        
    x_range=128
    ax.set_ylim(-128,128)
    for k in range(-128,128,8):
        ax.plot([0,x_range],[k,k], 'k-', alpha=.1, linewidth=1)
    ln=[]
    labels=['I_O','Q_O','IB_O','IQ_O','I_E','Q_E','IB_E','IQ_E']
    avgSubAdc=[]
    minSubAdc=[]
    maxSubAdc=[]
    for subADC in range(8):
        avgSubAdc.append(np.mean(capture[subADC::8]))
        minSubAdc.append(np.min(capture[subADC::8]))
        maxSubAdc.append(np.max(capture[subADC::8]))
        ln.append(ax.plot(capture[subADC:x_range*8:8],'o',markersize=2, label=labels[subADC], alpha=.8))
    lnAvg, = ax.plot([0,x_range],[avgCap,avgCap], 'r--', alpha=.5, linewidth=1)
    lnMin, = ax.plot([0,x_range],[minCap,minCap], 'y--', alpha=.8, linewidth=1)
    lnMax, = ax.plot([0,x_range],[maxCap,maxCap], 'y--', alpha=.8, linewidth=1)
    lnZero, = ax.plot([0,x_range],[0,0], 'g--', alpha=.8, linewidth=1)
    strAvg=''
    for avg1,min1,max1 in zip(avgSubAdc,minSubAdc,maxSubAdc):
        strAvg+='{0:6.2f} {1:6.2f} {2:6.2f} {3:6.2f}\n'.format(avg1,min1,max1,min1+max1)
    textstr = 'avg:{0:6.2f} min:{1} max:{2}\n{3}'.format(avgCap,minCap,maxCap,strAvg)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.65, textstr, transform=ax.transAxes, fontsize='large', verticalalignment='top', bbox=props)
    ax.set_title(fname, fontsize='large')
    ax.legend(loc='upper right', ncol=2)
    ax.yaxis.set_major_locator(plt.MultipleLocator(8))
    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    if ax == None:
        return fig
    else:
        return ax
    
def plotBV(fig=None, ax=None, capture=None, fname=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)

    bits=len(capture)/2/64
    avgCap = np.mean(capture)
    minCap = np.min(capture)
    maxCap = np.max(capture)

    result = bitVisDataSubADC(capture, NUM_CB=None, subADC=64)
    
    res=result[0]
    resBitSum=result[1]
    resDiff=result[2]
    resBitSumDiff=result[3]

    resBitSum1=[]
    for i in range(8):
        resBitSum1=resBitSum1+resBitSum[i::8]
    resBitSum=[]
    for i in range(4):
        resBitSum=resBitSum+resBitSum1[i*8:(i+1)*8]+resBitSum1[32+i*8:32+(i+1)*8]
    plt.gca().set_prop_cycle(None)
    ax.plot(resBitSum,'o')
    plt.gca().set_prop_cycle(None)
    for i in range(2):
        ax.plot([0,0],[0,0])
    for i in range(1,9):
        ax.plot([0,64],[(i)*bits,(i)*bits])
        
    maxArr=np.amax(resBitSum)
    textstr = 'avg:{0:6.2f} min:{1} max:{2}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    ax.set_title(fname)
    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    for i in range(64):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.1)
    for i in range(0,64,8):
        ax.plot([i,i],[0,maxArr],'k--',alpha=.3)
    for i in range(0,64,16):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.6)
    if ax == None:
        return fig
    else:
        return ax

def plotCB(fig=None, ax=None, capture=None, fname=None, log=False, zmaxIn=None , zminIn=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)

    avgCap = np.mean(capture)
    minCap = np.min(capture)
    maxCap = np.max(capture)

    z=[]
    for i in range(64):
        z.append(np.histogram(capture[i::64], bins=range(-128,128)))
    z1=[]
    for i in range(8):
        z1=z1+z[i::8]
    z=[]
    for i in range(4):
        z=z+z1[i*8:(i+1)*8]+z1[32+i*8:32+(i+1)*8]

    #y=[i for i in range(64)]*256
    #x=[i for i in range(-128,128)]*64
    #z=np.asarray(z).flatten()
    
    z1=[]
    for i in range(64):
        z1.append(z[i][0])
    z=np.asarray(z1,dtype='float').T
    
    if zmaxIn==None and zminIn==None:
        zmax=np.amax(z)
        zmin=0
        #print(zmin,zmax)
    else:
        zmax=zmaxIn
        zmin=zminIn
        
    if not log:
        for each in range(len(z)):

            for j in range(len(z[each])):
                if int(z[each][j])==0:
                    z[each][j]=np.nan

    extent=(0, 64, -128, 128)
    cmap = cm.jet
    for i in range(-128,128,8):
        ax.plot([0,64],[i,i],'k-',alpha=.3)
    for i in range(0,64,8):
        ax.plot([i,i],[-128,128],'k--',alpha=.3)
    for i in range(0,64,16):
        ax.plot([i,i],[-128,128],'k-',alpha=.6)

    ax.set_facecolor('whitesmoke')
    if log:
        im = plt.imshow(z[::-1], extent=extent, norm=LogNorm(vmin=1e-15, vmax=zmax), cmap=cmap, aspect='auto')
    else:
        im = plt.imshow(z[::-1], extent=extent, cmap=cmap, aspect='auto', vmin=zmin, vmax=zmax)

    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    ax.yaxis.set_major_locator(plt.MultipleLocator(8))

    plt.clim(zmin,zmax )
    #plt.colorbar(im,ax=ax)
    cbar=fig.colorbar(im,shrink=.3, aspect=5,ax=ax)
    ax.set_title(fname)
    textstr = 'avg:{0:6.2f} min:{1} max:{2}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    if ax == None:
        return fig
    else:
        return ax


def plotBV2(fig=None, ax=None, capture=None, fname=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)
    avgCap = np.mean(capture)
    minCap = int(np.min(capture ))
    maxCap = int(np.max(capture ))
    
    subADC=64
    NUM_CB=None
    if NUM_CB == None:
        lenCap = len(capture)
    else:
        lenCap = NUM_CB*256*64

    result = bitVisDataSubADC(capture, NUM_CB=NUM_CB, subADC=subADC, inv=False)
    exp_bits = int(len(capture)/2/subADC)
    res=result[0]
    resBitSum=result[1]
    resDiff=result[2]
    resBitSumDiff=result[3]
    cmap = cm.coolwarm#jet
    
    plotData=resBitSumDiff

    #y=[i for i in range(64)]*8
    #x=[i for i in range(8)]*64
    Z=(np.array(plotData) /[(lenCap/2/64*i+1) for i in range(8)] )*100

    plotDataSubADC = []
    for i in range(8):
        plotDataSubADC = plotDataSubADC + plotData[i::8]
    plotDataSubADC_1 = []
    for i in range(4):
        plotDataSubADC_1 = plotDataSubADC_1 + plotData[i*8:i*8+8] + plotData[i*8+32:i*8+32+8]
    plotDataSubADC=plotDataSubADC_1
    Z=(np.array(plotDataSubADC) /[(lenCap/2/64*i+1) for i in range(8)])*100

    surf = ax.imshow(Z.T, cmap=cmap, vmin=-10, vmax=10, aspect=4,)
    cbar=fig.colorbar(surf,shrink=.3, aspect=5,ax=ax)
    ax.set_title(fname)
    for i in range(0,64,8):
        ax.plot([i-.5,i-.5],[-0.5,7.5],'k--',alpha=.3)
    for i in range(0,64,16):
        ax.plot([i-.5,i-.5],[-0.5,7.5],'k-',alpha=.6)

    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    if ax == None:
        return fig
    else:
        return ax

def plotFolderReport(fnames):
    for fname in fnames:
        fig = plt.figure(figsize=(40,20))
        plt.tight_layout()
        ax=[]
        for i in range(5):
            ax.append(fig.add_subplot(2,3,i+1))
        capture    =cleanCapture(np.load(fname))
        plotTD(fig=fig, ax=ax[1], capture=capture, fname=fname)
        plotPlainTD(fig=fig, ax=ax[0], capture=capture, fname=fname)
        plotBV2(fig=fig, ax=ax[3], capture=capture, fname=fname)
        plotBV(fig=fig, ax=ax[2], capture=capture, fname=fname)
        plotCB(fig=fig, ax=ax[4], capture=capture, fname=fname, log=False)
        #plotCB(fig=fig, ax=ax[5], capture=capture, fname=fname, log=True)
        fig.savefig(fname.split('.npy')[0]+'_REPORT.png')
        plt.show()
        plt.clf()
        
def plotCaptureReport(capture,title):
    fig = plt.figure(figsize=(40,20))
    plt.tight_layout()
    ax=[]
    for i in range(5):
        ax.append(fig.add_subplot(2,3,i+1))
    plotTD(fig=fig,ax=ax[1], capture=capture, fname=title)
    plotPlainTD(fig=fig, ax=ax[0], capture=capture, fname=title)
    plotBV2(fig=fig, ax=ax[3], capture=capture, fname=title)
    plotBV(fig=fig, ax=ax[2], capture=capture, fname=title)
    plotCB(fig=fig, ax=ax[4], capture=capture, fname=title, log=False)
    return fig, ax

def convertAll(fnames):
    for fname in fnames:
        capture=np.loadtxt(fname)
        np.save(fname.split('.txt')[0]+'.npy', capture)

def plotCorners(fname, corners=12, types=[0,1,2,3,4]):
    folder = fname[0:fname.rfind('\\')]+'\\'
    conditions = corners
    
    fio=open(fname,'r')
    CAPTURES=fio.readlines()
    fio.close()

    header = CAPTURES[0].split(',')
    CAPTURES = CAPTURES[1:]
    pars=[]
    captures=[]
    for each in CAPTURES:
        each=each.split(',\n')[0]
        pars.append([each.split(',')[0]] + [float(i) for i in each.split(',')[1:7]])
        captures.append([int(i) for i in each.split(',')[7:]])

    for i in range(0,int(len(captures)/conditions)):
        fig=[]
        PARTS=[]
        for k in types:
            fig.append(plt.figure(figsize=(30,30)))
            plt.tight_layout()
            ax=[]
            for j in range(conditions):
                plotPos=[0,1,2,6,7,8,3,4,5,9,10,11]
                ax.append(fig[k].add_subplot(int(conditions/3),3,plotPos[j]+1))
                sample = i*conditions + j
                PART_NO = pars[sample][0]
                if k==0 and j==0:
                    PARTS.append(PART_NO)
                #PASS =  partPass(PART_NO.split('_ITER')[0])
                SUPPLY = pars[sample][1]
                AVDD10_ADC = pars[sample][2]
                TEMP = pars[sample][3]
                title='PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP)
                capture = captures[sample][7:]
                if k == 0:
                    plotTD(fig=fig[k], ax=ax[j], capture=capture, fname=title)
                if k == 1:
                    plotCB(fig=fig[k], ax=ax[j], capture=capture, fname=title)
                if k == 2:
                    plotBV(fig=fig[k], ax=ax[j], capture=capture, fname=title)
                if k == 3:
                    plotPlainTD(fig=fig[k], ax=ax[j], capture=capture, fname=title)
                if k == 4:
                    plotPlainBV2(fig=fig[k], ax=ax[j], capture=capture, fname=title)
            if k == 0:
                plt.savefig(folder+PARTS[-1]+'_TD.png')
            if k == 1:
                plt.savefig(folder+PARTS[-1]+'_CB.png')
            if k == 2:
                plt.savefig(folder+PARTS[-1]+'_BV.png')
            if k == 3:
                plt.savefig(folder+PARTS[-1]+'_PlainTD.png')
            if k == 4:
                plt.savefig(folder+PARTS[-1]+'_BV2.png')
            plt.show()
            plt.clf()

#def plotCsv(fname, types=[0], part='All', captureStart=7, save=False, SAMPLES=1024):
#    folder = fname[0:fname.rfind('\\')]+'\\'
#    
#    fio=open(fname,'r')
#    CAPTURES=fio.readlines()
#    fio.close()
#
#    header = CAPTURES[0].split(',')
#    CAPTURES = CAPTURES[1:]
#    pars=[]
#    captures=[]
#    for each in CAPTURES:
#        each=each.split(',\n')[0]
#        pars.append([each.split(',')[0]] + [float(i) for i in each.split(',')[1:7]])
#        captures.append([int(i) for i in each.split(',')[captureStart:]])
#
#    for i in range(0,int(len(captures))):
#        sample = i
#        PART_NO = pars[sample][0]
#        SUPPLY = pars[sample][1]
#        AVDD10_ADC = pars[sample][2]
#        TEMP = pars[sample][3]
#        if part=='All' or part==PART_NO:
#            for k in types:
#                fig=plt.figure(figsize=(16,8))
#                ax=fig.add_subplot(1,1,1)
#                title='PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP)
#                capture = captures[sample][7:]
#                if k == 0:
#                    plotTD(fig=fig, ax=ax, capture=capture, fname=title)
#                    if save: plt.savefig(folder+PART_NO+'_TD_{}.png'.format(time.time()))
#                if k == 1:
#                    plotCB(fig=fig, ax=ax, capture=capture, fname=title)
#                    if save: plt.savefig(folder+PART_NO+'_CB_{}.png'.format(time.time()))
#                if k == 2:
#                    plotBV(fig=fig, ax=ax, capture=capture, fname=title)
#                    if save: plt.savefig(folder+PART_NO+'_BV_{}.png'.format(time.time()))
#                if k == 3:
#                    plotPlainTD(fig=fig, ax=ax, capture=capture, fname=title, SAMPLES=SAMPLES)
#                    if save: plt.savefig(folder+PART_NO+'_PlainTD_{}.png'.format(time.time()))
#                plt.show()
#                plt.clf()
    
def plotCsv(fname, types=[0], part='All', captureStart=7, saveEachFig=False, SAMPLES=1024, pdf=True):
    suffix=['TD','CB','BV','PlaingTD']
    suffixString=[]
    pdfName=fname.split('.csv')[0]+'.pdf'
    with PdfPages(pdfName) as pdf:
        folder = fname[0:fname.rfind('\\')]+'\\'
        fio=open(fname,'r')
        header = fio.readline().split(',')
        each=fio.readline()
        while each  != '':
            par=[each.split(',')[0]] + [float(i) for i in each.split(',')[1:7]]
            capture=[int(i) for i in each.split(',')[captureStart:-1]]
            PART_NO = par[0]
            SUPPLY = par[1]
            AVDD10_ADC = par[2]
            TEMP = par[3]
            if part=='All' or part==PART_NO:
                for k in types:
                    fig=plt.figure(figsize=(16,8))
                    ax=fig.add_subplot(1,1,1)
                    title='PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP)
                    capture = capture[7:]
                    if k == 0:
                        plotTD(fig=fig, ax=ax, capture=capture, fname=title)
                        if saveEachFig: 
                            plt.savefig(folder+PART_NO+'_TD_{}.png'.format(time.time()))
                    if k == 1:
                        plotCB(fig=fig, ax=ax, capture=capture, fname=title)
                        if saveEachFig: 
                            plt.savefig(folder+PART_NO+'_CB_{}.png'.format(time.time()))
                    if k == 2:
                        plotBV(fig=fig, ax=ax, capture=capture, fname=title)
                        if saveEachFig: 
                            plt.savefig(folder+PART_NO+'_BV_{}.png'.format(time.time()))
                    if k == 3:
                        plotPlainTD(fig=fig, ax=ax, capture=capture, fname=title, SAMPLES=SAMPLES)
                        if saveEachFig: 
                            plt.savefig(folder+PART_NO+'_PlainTD_{}.png'.format(time.time()))
                    if pdf: pdf.savefig(fig)
                    plt.show()
                    plt.clf()
                    plt.close()
            each=fio.readline()

    fio.close()
    
def plotCsvReport(fname, types=[0], part='All', captureStart=7, saveEachFig=False, SAMPLES=1024, pdf=True, parStart=1):
    pdfName=fname.split('.csv')[0]+'.pdf'
    with PdfPages(pdfName) as pdf:
        folder = fname[0:fname.rfind('\\')]+'\\'
        fio=open(fname,'r')
        header = fio.readline().split(',')
        each=fio.readline()
        while each  != '':
            par=[each.split(',')[0]] + [float(i) for i in each.split(',')[1:7]]
            print(par)
            capture=[int(i) for i in each.split(',')[captureStart:-1]]
            PART_NO = par[0]
            SUPPLY = par[parStart+0]
            AVDD10_ADC = par[parStart+1]
            TEMP = par[parStart+2]
            title='PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP)
            fig, ax=plotCaptureReport(capture,title)
            pdf.savefig(fig)
            #plt.show()
            plt.clf()
            plt.close()
            each=fio.readline()

    fio.close()
            
    

In [ ]:
FOLDER = ('.\\CAPTURES_PVT\\BHASKAR\\SC\\')
fnames=glob.glob(FOLDER+'*.csv')
for fname in fnames:
    plotCsvReport(fname, types=[0], part='All', captureStart=7, parStart=1, saveEachFig=False, SAMPLES=1024, pdf=True)

In [ ]:
FOLDER = ('.\\CAPTURES_PVT\\BHASKAR\\SC\\4_26\\')
fnames=glob.glob(FOLDER+'*.csv')
for fname in fnames:
    plotCsvReport(fname, types=[0], part='All', captureStart=16, parStart=10, saveEachFig=False, SAMPLES=1024, pdf=True)

In [ ]:
FOLDER=r'.\\SC\\with-FW_Fin-MXG_captures\\'
FOLDER=r'.\\SC\\with-FW_Fin-Prism_captures\\'
FOLDER=r'.\\SC\\ADC_Char\\B009_08_2018c\\'
FOLDER=r'.\\SC\\PreFIB\\'

FOLDERS=[]
FOLDERS+=[r'.\\SC\\PostFIB\\']
FOLDERS+=[r'.\\SC\\TP100c_ref_2\\']
FOLDERS+=[r'.\\SC\\TP100c_ref\\']
FOLDERS+=[r'.\\SC\\SS_AMS\\']
FOLDERS+=[r'.\\SC\\APPS\\']
FOLDERS+=[r'.\\190416_ADC_SQW\\']

FOLDERS=[r'.\\SC\\Bench123_with-FW_Fin-Prism_captures\\']
for FOLDER in FOLDERS:
    fnames=glob.glob(FOLDER+'*.txt')
    convertAll(fnames)
for FOLDER in FOLDERS:
    fnames=glob.glob(FOLDER+'*.npy')
    plotAll(fnames)

In [ ]:
FOLDERS=[]
FOLDERS+=[r'.\\SC\\TP100c_ref_2\\']
FOLDERS+=[r'.\\SC\\TP100c_ref\\']
FOLDERS+=[r'.\\SC\\SS_AMS\\']
FOLDERS+=[r'.\\SC\\APPS\\']

FOLDERS=[r'.\\CAPTURES_PVT\\BHASKAR\\1stPASS\\Andover-PScrBench-Val Brd_A001_with-FW_Fin-SigGen(530Mhz)_captures\\']
for FOLDER in FOLDERS:
    fnames=glob.glob(FOLDER+'*.txt')
    for fname in fnames:
        capture=np.loadtxt(fname)
        np.save(fname.split('.txt')[0]+'.npy', capture)
    fnames=glob.glob(FOLDER+'*.npy')
    plotFolderReport(fnames)
#

In [ ]:
#FOLDER = ('.\\CAPTURES_PVT\\BHASKAR\\SIN\\')
#fname = FOLDER+'SG_SIN530_SQUARES_ANDOVER_1556056080.44.csv'
FOLDER = ('.\\CAPTURES_PVT\\BHASKAR\\SQUARES\\')
fname = FOLDER+'PRISM_SQUARES_SQUARES_ANDOVER_1556072663.72.csv'
fname = FOLDER+'SQUARESSQUARES_ANDOVER_1556132478.43.csv'
#FOLDER = ('.\\NOISE\\')
#fname = FOLDER+'NOISE_12CORNERS_1556203897.19.csv'



plotCorners(fname, corners=12, types=[0])

In [ ]:
FOLDER = ('.\\POWER\\')
fname = FOLDER+'POWER_LOG_LINK_CAPTURE_PVT_Andover_Part_TS_1556121560.24.csv'
FOLDER = ('.\\CAPTURES_PVT\\BHASKAR\\SC\\')
fname = FOLDER+'Prisn_SQRWave531M-VDD0p8V+15%_VMargin_T85CSQUARES_SCC_BENCH123_1556245915.83.csv'


plotCsv(fname,  types=[1], part='SC_FFF_96', captureStart=7+83, save=True)

In [ ]:
FOLDER = ('.\\LOGS\\')
fname = FOLDER+'Prisn_SQRWave531M_V_T25CSQUARES_ANDOVER_1556149163.91.csv'
FOLDER = ('.\\CAPTURES_PVT\\BHASKAR\\SC\\')
#fname = FOLDER+'Prisn_SQRWave531M-VDD0p8V+15%_VMargin_T85CSQUARES_SCC_BENCH123_1556245915.83.csv'
#fname = FOLDER+'Prisn_SQRWave531M-VDD0p8V_VMargin_T25CSQUARES_SCC_BENCH123_1556234748.72.csv'
fname = FOLDER+'Pris_SQR531MHz_VTSQUARES_ANDOVER_1556217202.2.csv'


plotCsv(fname, types=[2], part='All', SAMPLES=4096)


In [ ]:
FOLDER = ('.\\LOGS\\')
fname = FOLDER+'Prisn_SQRWave531M_V_T25CSQUARES_ANDOVER_1556149163.91.csv'
plotCsv(fname, type=[1], part='TTT_226')